In [5]:
import json

import torch
from PIL import Image
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import NeptuneLogger
from torch import nn
from torchvision import transforms as T
from typing import List
from config import FT_NET_CFG, SHAPE_EMBEDDING_CFG
from src.models.baseline import LitModule
from src.datasets.get_loader import get_train_loader
from tqdm.auto import tqdm

In [6]:
train_loader = get_train_loader()

In [7]:
net = LitModule(shape_edge_index=SHAPE_EMBEDDING_CFG.EDGE_INDEX,
                shape_pose_n_features=SHAPE_EMBEDDING_CFG.POSE_N_FEATURES,
                shape_n_hidden=SHAPE_EMBEDDING_CFG.N_HIDDEN,
                shape_out_features=SHAPE_EMBEDDING_CFG.OUT_FEATURES,
                shape_relation_layers=SHAPE_EMBEDDING_CFG.RELATION_LAYERS,
                class_num=751,
                r50_stride=FT_NET_CFG.R50_STRIDE,
                r50_pretrained_weight=FT_NET_CFG.PRETRAINED)


In [ ]:
neptune_logger = NeptuneLogger(
    project="work_space/save/",
    api_key=
    "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vbmV3LXVpLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI2MWM3OWRkYi0yMTFlLTQzNjMtOGEzOS0yOGI0MjUxNmRkNjkifQ==",
    log_model_checkpoints=False,
)

In [8]:
trainer = Trainer(accelerator='gpu', max_epochs=10)

trainer.fit(model=net, train_dataloaders=train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type           | Params
-----------------------------------------------------
0 | ft_net            | FTNet          | 25.6 M
1 | shape_embedding   | ShapeEmbedding | 300 K 
2 | fusion            | FusionNet      | 2     
3 | id_classification | Linear         | 769 K 
-----------------------------------------------------
1.1 M     Trainable params
25.6 M    Non-trainable params
26.6 M    Total params
106.511   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

In [ ]:
with open('data/cuhk03/jsons/train.json', 'rb') as f:
    img_list = json.load(f)

import random  


# def get_img_tensor(img_path):
#     img = Image.open(img_path)
#     img_tensor = transforms(img)
#     return img_tensor


# def get_pose_tensor(pose: List[List[float]]):
#     return Tensor(pose)

In [ ]:
def getitem(img_list):
    failed = []
    for a_img in img_list:
        a_img_path = a_img['img_path']
        a_id = a_img['p_id']
        a_orientation = a_img['orientation']
        a_pose = a_img['pose_landmarks']

        same_id, diff_id = [], []
        for item in img_list:
            if item['p_id'] == a_id:
                same_id.append(item)
            else:
                diff_id.append(item)

        same_id_diff_ori, diff_id_same_ori = [], []

        if 0 <= a_orientation <= 9 or 63 <= a_orientation <= 71 or 27 <= a_orientation <= 45:
            # anchor has back or front orientation
            for item in same_id:
                if 45 <= item["orientation"] < 63 or 9 <= item["orientation"] < 27:
                    # found positive sample of sideway orientation
                    same_id_diff_ori.append(item)
            for item in diff_id:
                if 0 <= item["orientation"] <= 9 or 63 <= item["orientation"] <= 71 or 27 <= item["orientation"] <= 45:
                    diff_id_same_ori.append(item)
        else:
            # anchor has sideway orientation
            for item in same_id:
                if 0 <= item["orientation"] <= 9 or 63 <= item["orientation"] <= 71 or 27 <= item["orientation"] <= 45:
                    # found positive sample of back orientation
                    same_id_diff_ori.append(item)
            for item in diff_id:
                if 45 <= item["orientation"] < 63 or 9 <= item["orientation"] < 27:
                    diff_id_same_ori.append(item)

        # if len(diff_id_same_ori) == 0:
        #     print(
        #         a_img_path
        #     )
    
        try:
            p_img = random.choice(same_id_diff_ori)
            n_img = random.choice(diff_id_same_ori)
        except: 
            failed.append(a_id)

    return [*set(failed)]
        # p_img_path = p_img['img_path']
        # p_pose = p_img['pose_landmarks']

        
        # n_img_path = n_img['img_path']
        # n_pose = n_img['pose_landmarks']

        # a_img_tensor = get_img_tensor(a_img_path)
        # p_img_tensor = get_img_tensor(p_img_path)
        # n_img_tensor = get_img_tensor(n_img_path)
        # a_pose_tensor = get_pose_tensor(a_pose)
        # p_pose_tensor = get_pose_tensor(p_pose)
        # n_pose_tensor = get_pose_tensor(n_pose)
         #(a_img_tensor, p_img_tensor, n_img_tensor), (a_pose_tensor, p_pose_tensor, n_pose_tensor), a_id


In [ ]:
failed = getitem(img_list)


In [ ]:
len(failed)

43

In [ ]:

failed[:5]

['/media/dustin/DATA/Research/2DReID/SMPLMarket/bounding_box_train/0007/0007_c1s6_028546_01.jpg',
 '/media/dustin/DATA/Research/2DReID/SMPLMarket/bounding_box_train/0007/0007_c1s6_028546_04.jpg',
 '/media/dustin/DATA/Research/2DReID/SMPLMarket/bounding_box_train/0007/0007_c2s3_071002_01.jpg',
 '/media/dustin/DATA/Research/2DReID/SMPLMarket/bounding_box_train/0007/0007_c2s3_071052_01.jpg',
 '/media/dustin/DATA/Research/2DReID/SMPLMarket/bounding_box_train/0007/0007_c3s3_077419_03.jpg']